In [ ]:
import numpy as np
import torch
import matplotlib.pyplot as plt
from model import LSTM
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader, TensorDataset

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

### **Load the data**

In [ ]:
data = pd.read_csv('data/solar_wind_parameters_data_1_hourly_all.csv')

data = data.drop(columns=['Unnamed: 0','Timestamp'])

# data.head()

features = ['Scalar_B', 'BZ_GSE', 'BZ_GSM', 'Proton_Density', 'SW_Plasma_Speed']
target = ['Dst-index']

batch_size = 32

# Select the features and target
data = data[features + target]

# Normalize the data
scaler = MinMaxScaler()
data_scaled = scaler.fit_transform(data)

# Split the data into sequences
def sequence(data, seq_length):
    X, y = [], []
    for i in range(len(data) - seq_length):
        X.append(data[i:i+seq_length, :-1]) # all columns except the last
        y.append(data[i+seq_length, -1])  # target is the last column
        # print(f'X:{np.array(X)}, Y:{np.array(y)}')
    return np.array(X), np.array(y)

SEQ_LENGTH = 24
X, y = sequence(data_scaled, SEQ_LENGTH)

# Convert to PyTorch tensors
X_tensor = torch.tensor(X, dtype=torch.float32).to(device)
y_tensor = torch.tensor(y, dtype=torch.float32).to(device)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_tensor, y_tensor, test_size=0.2, shuffle=False)
print(f'Train shape: {X_train.shape}, {y_train.shape}')
print(f'Test shape: {X_test.shape}, {y_test.shape}')

test_data = TensorDataset(X_test, y_test)
test_load = DataLoader(test_data, batch_size=batch_size, shuffle=False)

### **Load the Model**

In [ ]:
# Load the trained model
path = 'results/run3/best_model.pth'
checkpoint = torch.load(f'{path}', map_location=device)
print(checkpoint.keys())

# Recreate the model
model = LSTM(checkpoint['input_size'], checkpoint['hidden_layer_size'], output_size=1).to(device)

# Load the model weights
model.load_state_dict(checkpoint['model_state_dict'])
model.eval()  # Set to evaluation mode

print("Model loaded successfully!")

### **Plot**

In [ ]:
X_test, y_test = next(iter(test_load))  # Get one batch from test dataset
X_test = X_test.to(device)

# Predict
with torch.no_grad():
    y_pred = model(X_test).cpu().numpy()  # Convert to NumPy for plotting
    # print(f'y_pred:{y_pred.shape} {y_pred}')

# Convert true values to NumPy
y_test = y_test.cpu().numpy()

# Fix the shape by flattening y_pred
y_pred = y_pred.reshape(y_pred.shape[0], -1).mean(axis=1)  # Average across time steps if needed
print(f'y_pred:{y_pred.shape} {y_pred}')


In [ ]:
plt.figure(figsize=(10,5))
plt.plot(y_test, label="Actual DST", linestyle='-', marker='o', alpha=0.7)
plt.plot(y_pred, label="Predicted DST", linestyle='--', marker='x', alpha=0.7)

plt.xlabel("Time Step")
plt.ylabel("DST Index")
plt.title("DST Index Prediction using LSTM")
plt.legend()
plt.grid()
plt.show()